In [2]:
import pandas as pd
import csv
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


In [3]:
def cleanBazaarData():
    with open("bazaarData.txt", 'r') as file:
        reader = csv.reader(file, delimiter=';')
        dataset = {}
        n = 0



        for line in reader:
            di = eval(line[1])
            di.update({'product': line[0]})

            reliability = 0
            volumePerc = (di['sellMovingWeek'] / (
                        di['buyMovingWeek'] + di['sellMovingWeek'])) >= .40
            boolWeekly = di['sellMovingWeek'] >= 150000 and di['buyMovingWeek'] >= 150000
            boolOrder = di['sellOrders'] >= 40 and di['buyOrders'] >= 40
            # Determine Weights (A lot of trial and error)

            if di['RoI'] >= 5:
                reliability += 3
                if di['margin'] >= 40:
                    reliability += 3
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                elif 15 < di['margin'] < 40:
                    reliability += 2
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2

            elif 0 < di['RoI'] < 5:
                reliability += 1
                if di['margin'] >= 40:
                    reliability += 3
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                elif 15 < di['margin'] < 40:
                    reliability += 2
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2

            else:
                # print(product)
                continue
            di['reliability'] = reliability
            dataset.update({n: di})
            n += 1

        for k in dataset:
            if dataset[k]['reliability'] >= 6:
                dataset[k]['reliability'] = 1
            else:
                dataset[k]['reliability'] = 0




        df = pd.DataFrame.from_dict(dataset,orient='index', columns=['product','buyVolume', 'buyOrders', 'buyPrice', 'buyMovingWeek', 'sellVolume',
                                                      'sellOrders', 'sellPrice', 'sellMovingWeek', 'RoI', 'margin','buySellDiff','reliability'])

        cleanDF = df.drop(columns=['product'])

        

        return cleanDF.drop_duplicates()


data = cleanBazaarData()


In [ ]:
# As we can see when running the code below, a logistic model is not the best option for our dataset.
# Had to bump max_iter up quite a bit for convergence to happen. and it's accuracy while decent is not.
# What I feel is capable of our data.

In [7]:
y = data['reliability']
X = data.drop(columns=['reliability'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)


LRclass = LogisticRegression(random_state=40, solver="lbfgs", penalty="l2", max_iter=1500)
LRclass.fit(X_train, y_train)
pred = LRclass.predict(X_test)

LRclassCFM = confusion_matrix(y_test, pred)
print(round((LRclassCFM[0][0]/(LRclassCFM[0][0]+LRclassCFM[1][0]))*100, 2), '% Correct Predictions. Logistic Regression Classifier')


89.0 % Correct Predictions. Logistic Regression Classifier
